In [ ]:
--Austin's Shit



In [37]:
require('hdf5')
require 'nn'
require 'optim'
require 'math'

f = hdf5.open("SST2.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
Y_valid = f:read("valid_output"):all()
--X_test = f:read("test_input"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

# Doc by word

In [38]:
--DOC BY WORD (vincent's stuff)
--feature weight: counts
function createDocWordMatrix(vocab, max_sent_len, sparseMatrix)
    docword = torch.zeros(sparseMatrix:size(1), vocab)
    for i=1,sparseMatrix:size(1) do
        for j=1, max_sent_len do
            local idx = (sparseMatrix[i][j])
            if idx ~= 0 then
                docword[i][idx] = 1 --+ docword[i][idx]
            end
        end        
    end
    return docword
end

function onehotencode(classes, target)
    onehot = torch.zeros(target:size(1), classes)
    for i=1,target:size(1) do
        onehot[i][target[i]] = 1
    end
    return onehot
end

In [39]:
X_train=createDocWordMatrix(nfeatures, 53, X_train)
X_valid=createDocWordMatrix(nfeatures, 53, X_valid)

In [40]:
W0 = torch.randn(5,nfeatures)*.01
b0 =torch.zeros(5,1)*.001

In [5]:
X_toy = torch.Tensor({{1,0,0},{0,1,0},{0,0,1},{1,0,0},{1,0,0},{0,1,0},{0,0,1},{1,0,0},{1,0,0},{0,1,0},{0,0,1},{1,0,0}})
Y_toy = torch.Tensor({1,2,3,1,1,2,3,1,1,2,3,1})
W_toy = torch.randn(5,3)
b_toy = torch.zeros(5,1)*.001
print(X_toy)

In [ ]:
print(X_valid:sub(1,1,1,50))


In [90]:
function isnan(m)
    if m ~= m then
        return true
    else
        return false
    end
end

function linear(b, W, x)
    --weight matrix W (dout x din), bias b (dout x 1), input matrix x (n x din)
    local B = b:clone()
    local B = B:expand(B,B:size(1),x:size(1))
    return ((W*x:t()):add(B)):t()
end
    
function second_max(Y_pred, Y_true)
    --Y_pred is a nx5 tensor
    local second_maxes = torch.zeros(Y_pred:size(1),1)
    local indices = second_maxes:clone()
    local Y_prime = (Y_pred):clone()
    for i = 1, Y_pred:size(1) do
        Y_prime[i][Y_true[i]] = -100
        z1,z2 = torch.max(Y_prime[i],1)
        second_maxes[i] = z1[1]
        indices[i] = z2[1]
    end
    return second_maxes, indices
end

function correct_val(Y_pred, Y_true)
    return_val = torch.zeros(Y_true:size(1),1)
    for i = 1, Y_true:size(1) do
        return_val[i] = Y_pred[i][Y_true[i]]
    end
    return return_val
end

function ReLU(input)
    for i = 1,input:size(1) do
        if input[i][1]<0 then
            input[i][1] =0
        end
    end
    return input
end

function hinge_loss(Y_pred, Y_true, W,lambda,b, mode)
    --Y_pred is a nx5 tensor, Y_true is an nx1 tensor
    --mode = 'vanilla' for vanilla hinge
    --mode = 'L2' for hinge^2
    if mode == 'vanilla' then
        reg = 1
    elseif mode == 'squared' then
        reg = 2
    end
    
    params = torch.cat(W,b,2)
    local ones_vector = torch.ones(Y_true:size(1),1)
    local loss_vector = torch.pow(ReLU(ones_vector+second_max(Y_pred, Y_true)-correct_val(Y_pred,Y_true)),reg)
    loss1 = lambda*torch.pow((torch.norm(params)),2)
    loss2 = loss_vector:sum(1)[1][1]
    
    return (loss1 + loss2)
end

function hinge_grad(W,x,b,Y_true,lambda, mode)
    local Y_pred = linear(b,W,x)
    local loss = hinge_loss(Y_pred, Y_true, W, lambda, b, mode)
    local condition = torch.Tensor(Y_true:size(1),1)
    local correct = correct_val(Y_pred,Y_true) 
    local second, s_indices = second_max(Y_pred,Y_true)
    local condition = correct - second
    local W_grad = torch.zeros(W:size())
    local W_temp = torch.zeros(W:size())
    local b_grad = torch.zeros(b:size())
    local b_temp = torch.zeros(b:size())
    local debug = torch.zeros(4)
    for i = 1, condition:size(1) do
        if condition[i][1] < 1 then
            for j = 1, W:size(1) do
                if j == Y_true[i] then
                    b_temp[j] = -1
                    W_temp[j] = x[i]*(-1)*loss
                    debug[1] = debug[1] + 1
                elseif j == s_indices[i][1] then
                    debug[2] = debug[2] + 1
                    b_temp[j] = 1
                    W_temp[j] = x[i]*loss
                else
                    debug[3] = debug[3] + 1
                    b_temp[j] = 0
                    W_temp[j] = W_temp[j]*0
                end
            end
            b_grad:add(b_temp)
            W_grad:add(W_temp)
        else
            debug[4] = debug[4] + 1
            b_temp = torch.zeros(b_temp:size())
            W_temp = torch.zeros(W_temp:size())
        end
    end
    return W_grad:add(2*lambda,W),b_grad:add(2*lambda,b)
    --return W_grad,0--b_grad
end



function obj(params,X,Y,batch,lambda, mode)
    if batch > 0 then
        bsize= batch
        local idx = torch.randperm(X:size(1)):sub(1,bsize)

        x = torch.Tensor(bsize, X:size(2))

        Y = Y:reshape(Y:size(1), 1)

        y = torch.Tensor(bsize, 1)

        for i=1,bsize do
            x[i] = X[idx[i]]
            y[i] = Y[idx[i]]
        end
        
        y = y:squeeze()
    else
        y = Y
        x = X
    end
    
    local w = (params:sub(1,params:size(1),1,params:size(2)-1))
    
   
    b = params:sub(1,params:size(1),params:size(2),params:size(2))
    local ypred = linear(b,w,x)
    local loss = hinge_loss(ypred,y,w,lambda,b, mode)
    
    local w1,b1 = hinge_grad(w,x,b,y,lambda, mode)
    local grads = torch.zeros(params:size())
    grads[{{},{1,params:size(2)-1}}] = w1
    grads[{{},{params:size(2),params:size(2)}}] = b1
    return loss, grads
end

function obj_final(params)
    
    loss,grads = obj(params, X_train, Y_train, 500, 1, 'vanilla')
    print(loss)
    return loss,grads
end

In [9]:
p0 = torch.cat(W0,b0,2)
_,_,n = optim.checkgrad(obj_final,p0)
print(n:size())

invalid arguments: nil nil number 
expected arguments: [*DoubleTensor*] DoubleTensor DoubleTensor [index] | [*DoubleTensor*] {DoubleTensor+} [index]
stack traceback:
	[C]: at 0x060c5250
	[C]: in function 'cat'
	[string "p0 = torch.cat(W0,b0,2)..."]:1: in main chunk
	[C]: in function 'xpcall'
	...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:179: in function <...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:143>
	...s/austinshin/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0105c52bc0: 

In [8]:
toy_pred = linear(b_toy,W_toy,X_toy)

toy_second, toy_indices = second_max(toy_pred, Y_toy)
loss = hinge_loss(toy_pred, Y_toy, W_toy, 0,b_toy, 'squared')
print(loss)

116.17501506821	


In [74]:
m = torch.pow(torch.ones(100)*2,2)
p0 = torch.cat(W0,b0,2)

In [100]:
--p0 = torch.cat(W0,b0,2)
local state = {maxIter = 1, learningRate = .001, evalCounter = 1}
d = torch.Tensor(50)
for i = 1,d:size(1) do
    p0, m = optim.adam(obj_final,p0, state)
    d[i] = m[1]
end
print(d)

497.79249473163	


470.82573854668	


470.21417824026	


466.60985639752	


443.59150333072	


...

431.21034025116	


449.21542047801	


422.92486005714	


439.22632660758	


 497.7925
 470.8257
 470.2142
 466.6099
 443.5915
 481.0706
 474.8786
 474.0774
 475.8599
 459.1197
 454.7768
 464.8685
 422.1277
 451.4861
 460.4158
 441.4248
 426.3018
 464.2909
 451.8341
 458.8393
 438.6853
 437.4004
 428.5841
 419.2019
 435.6468
 437.0022
 435.6167
 420.3610
 420.3383
 427.4258
 451.1299
 472.3915
 410.4473
 431.5780
 429.4831
 440.6482
 441.1590
 439.1897
 460.7082
 441.7839
 427.4828
 434.3342
 445.7754
 426.3848
 438.1344
 436.0602
 431.2103
 449.2154
 422.9249
 439.2263
[torch.DoubleTensor of size 50]



In [101]:

 --intercept
b = p0:sub(1, p0:size(1), p0:size(2), p0:size(2))

--coefficients
W = p0:sub(1, p0:size(1), 1, p0:size(2)-1)

function predict(X, W, b)
    local b = b:t()
    return (X*W:t()):add(b:expand(b, X:size(1), b:size(2)))
end
function predict_score(ypred, ytrue)
    local c = 0
    for i=1,ypred:size(1) do
        if ypred[i][1] == ytrue[i] then
            c = c + 1       
        end
    end
    return c/ypred:size(1)
end


In [102]:
Y_pred = predict(X_valid, W, b)
_, score = torch.max(Y_pred, 2)
--_,Y_true = torch.max(Y_valid, 2)
acc_score = predict_score(score, Y_valid)
print(acc_score)

0.79816513761468	


In [ ]:
score

In [ ]:
print(W:sub(1,5,1,10))

In [ ]:
f,f_old = 1,2

In [ ]:
z = torch.sort(Y_train)

In [ ]:
z[40000]

In [ ]:
930-660

In [ ]:
z:size()

In [ ]:
70/156